## 1. Module

In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Remplacez 'chemin/vers/votre/projet/module' par le chemin absolu réel vers le dossier "module" via la commande pwd
module_path = '/Users/eliotsouthon/Desktop/POC/POC/Code/Modules'  
sys.path.append(module_path)
import ma_bibliotheque 

## 2. Data

In [3]:
pwd

'/Users/eliotsouthon/Desktop/POC/POC/Code'

In [4]:
csv_file_paths=ma_bibliotheque.path_data('/Users/eliotsouthon/Desktop/POC/POC/Data/Data_raw')
# Affiche la liste des chemins pour vérifier
print("Liste des chemins des fichiers CSV :")
for path in csv_file_paths:
    print(path)

Liste des chemins des fichiers CSV :
/Users/eliotsouthon/Desktop/POC/POC/Data/Data_raw/France/tmy_48.920_4.821_2005_2023.csv
/Users/eliotsouthon/Desktop/POC/POC/Data/Data_raw/France/w_51_raw.csv
/Users/eliotsouthon/Desktop/POC/POC/Data/Data_raw/France/consommation-quotidienne-brute-regionale.csv


In [5]:
# Load the data
file_path = csv_file_paths[1]  # Update this to the actual path if necessary
df_tempo = pd.read_csv(file_path, sep=';',encoding='utf-8') # Les 16 premieres lignes sont inutiles
print(df_tempo.shape,df_tempo.head())

(504523, 204)    NUM_POSTE NOM_USUEL        LAT    LON  ALTI  AAAAMMJJHH  RR1  QRR1  DRR1  \
0   51009001   ANGLURE  48.586333  3.819    77  2020020106  NaN   NaN   NaN   
1   51009001   ANGLURE  48.586333  3.819    77  2020020206  NaN   NaN   NaN   
2   51009001   ANGLURE  48.586333  3.819    77  2020020306  NaN   NaN   NaN   
3   51009001   ANGLURE  48.586333  3.819    77  2020020406  NaN   NaN   NaN   
4   51009001   ANGLURE  48.586333  3.819    77  2020020506  NaN   NaN   NaN   

   QDRR1  ...  INS  QINS  INS2  QINS2  TLAGON  QTLAGON  TVEGETAUX  QTVEGETAUX  \
0    NaN  ...  NaN   NaN   NaN    NaN     NaN      NaN        NaN         NaN   
1    NaN  ...  NaN   NaN   NaN    NaN     NaN      NaN        NaN         NaN   
2    NaN  ...  NaN   NaN   NaN    NaN     NaN      NaN        NaN         NaN   
3    NaN  ...  NaN   NaN   NaN    NaN     NaN      NaN        NaN         NaN   
4    NaN  ...  NaN   NaN   NaN    NaN     NaN      NaN        NaN         NaN   

   ECOULEMENT  QECOULEME

#### Where :
AAAAMMJJHH  : date de la mesure (annÃ©e mois jour heure)<br>
T : tempÃ©rature sous abri instantanÃ©e (en Â°C et 1/10)<br>
U : humiditÃ© relative (en %) <br>
GLO : rayonnement global horaire en heure UTC (en J/cm2)<br>
FF : force du vent moyennÃ© sur 10 mn, mesurÃ©e Ã  10 m (en m/s et 1/10)<br>


In [6]:
df = df_tempo[['AAAAMMJJHH', 'T', 'U', 'FF']].copy()

# Conversion en format datetime avec timezone

df['Date'] = pd.to_datetime(df['AAAAMMJJHH'], format='%Y%m%d%H%M')

df['Date'] = df['Date'].dt.strftime('%Y-%m-%dT%H:%M:%S+00:00')
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d %H:%M:%S%z')
df['Date'] = df['Date'].str.replace(r'(\+|-)(\d{2})(\d{2})$', r'\1\2:\3', regex=True)
df.drop(columns=["AAAAMMJJHH"], inplace=True)
df.head()


,T,U,FF,Date
0,NaN,NaN,NaN,2020-02-01 00:06:00+00:00
1,NaN,NaN,NaN,2020-02-02 00:06:00+00:00
2,NaN,NaN,NaN,2020-02-03 00:06:00+00:00
3,NaN,NaN,NaN,2020-02-04 00:06:00+00:00
4,NaN,NaN,NaN,2020-02-05 00:06:00+00:00


In [7]:
# Vérifier la présence de valeurs manquantes
def check_completeness(df):
    missing_count = df.isna().sum()
    total_rows = len(df)
    print("Analyse des valeurs manquantes par colonne :\n", missing_count)
    print("\nNombre total de lignes:", total_rows)
    print("\nLignes contenant des valeurs manquantes:", df.isna().any(axis=1).sum())
    if missing_count.sum() == 0:
        print("\nLe DataFrame est complet (aucune valeur manquante).")
    else:
        print("\nLe DataFrame contient des valeurs manquantes.")

check_completeness(df)

Analyse des valeurs manquantes par colonne :
 T         4872
U       294395
FF      321228
Date         0
dtype: int64

Nombre total de lignes: 504523

Lignes contenant des valeurs manquantes: 321262

Le DataFrame contient des valeurs manquantes.


In [8]:
print(df.columns)  # Affiche les colonnes du DataFrame

Index(['T', 'U', 'FF', 'Date'], dtype='object')


In [9]:
"""# S'assurer que la colonne 'Darte' est bien l'index
df.set_index('Date', inplace=True)

# Tracer les différentes séries temporelles avec des dates bien gérées
plt.figure(figsize=(14, 8))

# Tracer chaque série temporelle
for i, column in enumerate(['T', 'U', 'GLO', 'FF'], 1):
    plt.subplot(2, 2, i)
    plt.plot(df.index, df[column], label=column)
    plt.xlabel('Date')
    plt.ylabel(column)
    plt.title(f'{column} au fil du temps')
    plt.xticks(rotation=45)

    # Ajuster l'intervalle des dates pour qu'elles soient visibles sur l'axe des x
    plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d %H:%M'))
    plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.HourLocator(interval=24))  # Intervalle de 24 heures pour plus de lisibilité
    plt.gcf().autofmt_xdate()  # Rotation automatique des dates pour les rendre lisibles

    plt.legend()

plt.tight_layout()
plt.show()"""

"# S'assurer que la colonne 'Darte' est bien l'index\ndf.set_index('Date', inplace=True)\n\n# Tracer les différentes séries temporelles avec des dates bien gérées\nplt.figure(figsize=(14, 8))\n\n# Tracer chaque série temporelle\nfor i, column in enumerate(['T', 'U', 'GLO', 'FF'], 1):\n    plt.subplot(2, 2, i)\n    plt.plot(df.index, df[column], label=column)\n    plt.xlabel('Date')\n    plt.ylabel(column)\n    plt.title(f'{column} au fil du temps')\n    plt.xticks(rotation=45)\n\n    # Ajuster l'intervalle des dates pour qu'elles soient visibles sur l'axe des x\n    plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d %H:%M'))\n    plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.HourLocator(interval=24))  # Intervalle de 24 heures pour plus de lisibilité\n    plt.gcf().autofmt_xdate()  # Rotation automatique des dates pour les rendre lisibles\n\n    plt.legend()\n\nplt.tight_layout()\nplt.show()"

In [10]:

df.head()

,T,U,FF,Date
0,NaN,NaN,NaN,2020-02-01 00:06:00+00:00
1,NaN,NaN,NaN,2020-02-02 00:06:00+00:00
2,NaN,NaN,NaN,2020-02-03 00:06:00+00:00
3,NaN,NaN,NaN,2020-02-04 00:06:00+00:00
4,NaN,NaN,NaN,2020-02-05 00:06:00+00:00


In [11]:
numero_region=51
numero_pays=1
# Construire le nom du fichier en utilisant la première valeur de la colonne 'Code INSEE région'
nom_fichier = f"weather_{numero_pays}_{numero_region}.csv"
# Définir le chemin complet en joignant le chemin et le nom du fichier
chemin_complet = f"/Users/eliotsouthon/Desktop/POC/POC/Data/Data_silver/{nom_fichier}"

# Sauvegarder le DataFrame en CSV
df.to_csv(chemin_complet, index=False, sep=';', encoding='utf-8')